_author_: **Jimit Dholakia**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import json
import numpy as np
np.random.seed(105)

from tqdm.auto import tqdm
from pprint import pprint
from ast import literal_eval

In [3]:
import joblib
joblib.Parallel(n_jobs=-1)

Parallel(n_jobs=-1)

In [4]:
df = pd.read_csv('Job Information.csv')

df['Educational Levels'] = df['Educational Levels'].str.replace('\'', '\"')
df['Educational Levels'] = df['Educational Levels'].apply(json.loads)

df.loc[:,'Skills'] = df.loc[:,'Skills'].apply(lambda x: literal_eval(x))
# df['Skills'] = df['Skills'].apply(json.loads)
df['Skills'] = df['Skills'].apply(lambda x: ', '.join(map(str, x)))

In [5]:
df.head()

,Main Category,Sub Category,Sub Category Link,Job Title,Job Title Link,Salary,Salary Link,Average Salary,Educational Levels,No of Candidates,No of Jobs,Skills
0,Accounting and Financial Jobs,Accounting and Auditing,https://www.careerbuilder.com/browse/category/...,Account Manager,https://www.careerbuilder.com/jobs-account-man...,$73895,https://www.careerbuilder.com/salary-account-m...,"$75,500","{'VOCATIONAL': '$56,500', 'HIGH SCHOOL': '$55,...","1,664,228","16,704,086","Business Development, Negotiation, Customer Re..."
1,Accounting and Financial Jobs,Accounting and Auditing,https://www.careerbuilder.com/browse/category/...,Accountant,https://www.careerbuilder.com/jobs-accountant,$61416,https://www.careerbuilder.com/salary-accountant,"$76,000","{'VOCATIONAL': '$53,000', 'HIGH SCHOOL': '$54,...","204,372","2,239,450","Financial Transactions, Financial Statements, ..."
2,Accounting and Financial Jobs,Accounting and Auditing,https://www.careerbuilder.com/browse/category/...,Accounting,https://www.careerbuilder.com/jobs-accounting,$64548,https://www.careerbuilder.com/salary-accounting,"$72,000","{'VOCATIONAL': '$54,000', 'HIGH SCHOOL': '$52,...","3,384,564","44,464,804","Financial Transactions, Financial Statements, ..."
3,Accounting and Financial Jobs,Accounting and Auditing,https://www.careerbuilder.com/browse/category/...,Accounting Assistant,https://www.careerbuilder.com/jobs-accounting-...,$40171,https://www.careerbuilder.com/salary-accountin...,"$62,500","{'VOCATIONAL': '$47,000', 'HIGH SCHOOL': '$47,...","810,785","5,174,493","Financial Transactions, Accounting, Financial ..."
4,Accounting and Financial Jobs,Accounting and Auditing,https://www.careerbuilder.com/browse/category/...,Accounting Clerk,https://www.careerbuilder.com/jobs-accounting-...,$40207,https://www.careerbuilder.com/salary-accountin...,"$48,500","{'VOCATIONAL': '$42,500', 'HIGH SCHOOL': '$44,...","245,969","1,725,547","Financial Transactions, Financial Statements, ..."


In [6]:
df = pd.concat([df, pd.json_normalize(df['Educational Levels'])], axis=1)

df = df[['Job Title', 'Skills', 'VOCATIONAL', 'HIGH SCHOOL', 'ASSOCIATE', 'BACHELOR', 'MASTER', 'DOCTORATE', 'Average Salary']]
df.rename({'Average Salary': 'NOT SPECIFIED'}, axis=1, inplace=True)

In [7]:
df

,Job Title,Skills,VOCATIONAL,HIGH SCHOOL,ASSOCIATE,BACHELOR,MASTER,DOCTORATE,NOT SPECIFIED
0,Account Manager,"Business Development, Negotiation, Customer Re...","$56,500","$55,000","$59,500","$75,500","$87,500","$94,000","$75,500"
1,Accountant,"Financial Transactions, Financial Statements, ...","$53,000","$54,500","$57,500","$72,500","$80,000","$79,000","$76,000"
2,Accounting,"Financial Transactions, Financial Statements, ...","$54,000","$52,500","$57,000","$69,500","$79,500","$86,500","$72,000"
3,Accounting Assistant,"Financial Transactions, Accounting, Financial ...","$47,000","$47,500","$49,500","$60,500","$68,500","$77,000","$62,500"
4,Accounting Clerk,"Financial Transactions, Financial Statements, ...","$42,500","$44,500","$44,500","$50,000","$53,500","$60,000","$48,500"
...,...,...,...,...,...,...,...,...,...
1713,Truck Driving,,"$50,000","$52,000","$56,000","$67,000","$75,500","$81,000","$61,500"
1714,Truck Unloader,,"$41,500","$43,000","$42,500","$49,500","$49,500",NaN,"$43,000"
1715,Van Driver,,"$50,500","$49,500","$50,500","$53,500","$57,500","$57,000","$56,000"
1716,Warehouse Worker,"Entire Warehouse, General Office Associate, Ge...","$48,000","$47,000","$50,000","$58,000","$63,500","$69,500","$51,500"


In [8]:
df = df.melt(id_vars=['Job Title', 'Skills'], var_name='Education', value_name='Salary')
df.drop_duplicates(inplace=True)
df = df.sort_values(by=['Job Title', 'Education'])

In [9]:
df.sample(5)

,Job Title,Skills,Education,Salary
2727,Clinical Specialist,"Clinics, Acute Care, Diseases And Disorders, R...",HIGH SCHOOL,"$51,500"
4015,Clinical Educator,"Registered Nurse, Clinics, Licensed Practical ...",ASSOCIATE,"$71,000"
9421,Biotechnology,,DOCTORATE,"$97,500"
804,Production Worker,"Safety Knowledge, Assembling, Packing, Manufac...",VOCATIONAL,"$56,500"
2699,Phlebotomy Technician,"Training, Quality Control, Operations, Schedul...",HIGH SCHOOL,"$41,000"


In [10]:
df['Salary'] = df['Salary'].fillna('0')
df['Salary'] = df['Salary'].str.extract('(\d+.\d*)') # Get numbers from salary data eg. $123,456 --> 123456
df['Salary'] = df['Salary'].str.replace(",", "")
df['Salary'] = df['Salary'].astype('float')

df.dropna(subset=['Salary'], inplace=True)

In [11]:
df.head()

,Job Title,Skills,Education,Salary
4541,.Net Developer,"Technical Support, Configure System, Software ...",ASSOCIATE,101000.0
6259,.Net Developer,"Technical Support, Configure System, Software ...",BACHELOR,109000.0
9695,.Net Developer,"Technical Support, Configure System, Software ...",DOCTORATE,117500.0
2823,.Net Developer,"Technical Support, Configure System, Software ...",HIGH SCHOOL,104500.0
7977,.Net Developer,"Technical Support, Configure System, Software ...",MASTER,112500.0


In [12]:
df.shape

(9492, 4)

In [13]:
edu_dummies = pd.get_dummies(df['Education'], prefix='Education')
df = pd.concat([df, edu_dummies], axis=1)
del df['Education']
df.head()

,Job Title,Skills,Salary,Education_ASSOCIATE,Education_BACHELOR,Education_DOCTORATE,Education_HIGH SCHOOL,Education_MASTER,Education_NOT SPECIFIED,Education_VOCATIONAL
4541,.Net Developer,"Technical Support, Configure System, Software ...",101000.0,1,0,0,0,0,0,0
6259,.Net Developer,"Technical Support, Configure System, Software ...",109000.0,0,1,0,0,0,0,0
9695,.Net Developer,"Technical Support, Configure System, Software ...",117500.0,0,0,1,0,0,0,0
2823,.Net Developer,"Technical Support, Configure System, Software ...",104500.0,0,0,0,1,0,0,0
7977,.Net Developer,"Technical Support, Configure System, Software ...",112500.0,0,0,0,0,1,0,0


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor

In [16]:
from xgboost import XGBRegressor

In [17]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import math

In [18]:
df.columns

Index(['Job Title', 'Skills', 'Salary', 'Education_ASSOCIATE',
       'Education_BACHELOR', 'Education_DOCTORATE', 'Education_HIGH SCHOOL',
       'Education_MASTER', 'Education_NOT SPECIFIED', 'Education_VOCATIONAL'],
      dtype='object')

In [19]:
input_cols = ['Job Title', 'Skills', 'Education_ASSOCIATE', 'Education_BACHELOR',
       'Education_DOCTORATE', 'Education_HIGH SCHOOL', 'Education_MASTER',
       'Education_NOT SPECIFIED', 'Education_VOCATIONAL']

target_col = ['Salary']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(df[input_cols], df[target_col], test_size=0.1, random_state=4)

In [21]:
X_train

,Job Title,Skills,Education_ASSOCIATE,Education_BACHELOR,Education_DOCTORATE,Education_HIGH SCHOOL,Education_MASTER,Education_NOT SPECIFIED,Education_VOCATIONAL
5334,Mason,"Construction, Inspector, Renovation, Construct...",0,1,0,0,0,0,0
9062,Legal Consultant,"Joint Ventures, Corporate Laws, Divestitures, ...",0,0,1,0,0,0,0
9204,Science Teacher,"Differentiated Instruction, Secondary Educatio...",0,0,1,0,0,0,0
5496,Medical Secretary,"Administration, Maintain Indexed Record, Arran...",0,1,0,0,0,0,0
5638,Clerical,"Accounting, Financial Statements, Taxation, Au...",0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
142,Artist,"Zbrush, Figure Drawings, Concept Arts, 3d Art,...",0,0,0,0,0,0,1
5024,Outside Sales Representative,,1,0,0,0,0,0,0
3471,Banking,"Retail Banking, Unsecured Debt, Loan Originati...",1,0,0,0,0,0,0
10146,Store Manager,"Sales, Selling Techniques, Retail Management, ...",0,0,1,0,0,0,0


In [22]:
vectorizer1 = TfidfVectorizer()
vectorizer2 = TfidfVectorizer()

column_transformer = ColumnTransformer(
    [('tfidf1', vectorizer1, 'Job Title'),
    ('tfidf2', vectorizer2, 'Skills')],
    remainder='passthrough')

In [23]:
models = {
    'Linear Regression Regressor': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor(n_estimators=150),
    'Support Vector Regressor': SVR(),
    'SGD Regressor': SGDRegressor(max_iter=10000),
    'Elastic Net Regressor': ElasticNet(),
    'K Neighbors Regressor': KNeighborsRegressor(),
    'Extra Trees Regressor': ExtraTreesRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'XGBoost Regressor': XGBRegressor()
}

r2_scores = {}
rmse_scores = {}
result_df = pd.DataFrame()


In [24]:
models

{'Linear Regression Regressor': LinearRegression(),
 'Random Forest Regressor': RandomForestRegressor(n_estimators=150),
 'Support Vector Regressor': SVR(),
 'SGD Regressor': SGDRegressor(max_iter=10000),
 'Elastic Net Regressor': ElasticNet(),
 'K Neighbors Regressor': KNeighborsRegressor(),
 'Extra Trees Regressor': ExtraTreesRegressor(),
 'Gradient Boosting Regressor': GradientBoostingRegressor(),
 'XGBoost Regressor': XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
              sc

In [26]:
models_fitted = {}

In [27]:
for name, model in tqdm(models.items(), leave=False):
    print('Starting', name)
    pipe = Pipeline([
                      ('tfidf', column_transformer),
                      ('run_model', model)
                    ], verbose=True)
    pipe.fit(X_train, y_train)
    
#     scores[name] = pipe.score(X_test, y_test)
    
    y_pred = pipe.predict(X_test)
    r2_scores[name] = r2_score(y_test, y_pred)
    rmse_scores[name] = math.sqrt(mean_squared_error(y_test, y_pred))
    
    new_row = {'Model': name, 'R2 Score': r2_score(y_test, y_pred), 'RMSE': math.sqrt(mean_squared_error(y_test, y_pred))}
    
    result_df = result_df.append(new_row, ignore_index=True)
    
    models_fitted[name] = pipe
    
    print(f'{name} Complete')

  0%|          | 0/9 [00:00<?, ?it/s]

Starting Linear Regression Regressor
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing run_model, total=   1.5s
Linear Regression Regressor Complete
Starting Random Forest Regressor
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   0.3s
[Pipeline] ......... (step 2 of 2) Processing run_model, total= 4.1min
Random Forest Regressor Complete
Starting Support Vector Regressor
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing run_model, total=  13.4s
Support Vector Regressor Complete
Starting SGD Regressor
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing run_model, total=   6.4s
SGD Regressor Complete
Starting Elastic Net Regressor
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing run_model, total=   0.4s
Elast

In [28]:
pprint(r2_scores)

{'Elastic Net Regressor': 0.14738457286894113,
 'Extra Trees Regressor': 0.75446770816801,
 'Gradient Boosting Regressor': 0.7059070744289685,
 'K Neighbors Regressor': 0.6377276194896231,
 'Linear Regression Regressor': 0.8903221296001184,
 'Random Forest Regressor': 0.7647754708029897,
 'SGD Regressor': 0.88985000177259,
 'Support Vector Regressor': -0.014924644683672161,
 'XGBoost Regressor': 0.880311555633711}


In [29]:
sorted(r2_scores.items(), key=lambda x:x[1])

[('Support Vector Regressor', -0.014924644683672161),
 ('Elastic Net Regressor', 0.14738457286894113),
 ('K Neighbors Regressor', 0.6377276194896231),
 ('Gradient Boosting Regressor', 0.7059070744289685),
 ('Extra Trees Regressor', 0.75446770816801),
 ('Random Forest Regressor', 0.7647754708029897),
 ('XGBoost Regressor', 0.880311555633711),
 ('SGD Regressor', 0.88985000177259),
 ('Linear Regression Regressor', 0.8903221296001184)]

In [30]:
result_df

,Model,R2 Score,RMSE
0,Linear Regression Regressor,0.890322,6862.821021
1,Random Forest Regressor,0.764775,10050.426046
2,Support Vector Regressor,-0.014925,20876.614291
3,SGD Regressor,0.889850,6877.576274
4,Elastic Net Regressor,0.147385,19134.616459
5,K Neighbors Regressor,0.637728,12472.708734
6,Extra Trees Regressor,0.754468,10268.274665
7,Gradient Boosting Regressor,0.705907,11237.907604
8,XGBoost Regressor,0.880312,7169.176614


In [31]:
result_df.sort_values(by=['RMSE'])

,Model,R2 Score,RMSE
0,Linear Regression Regressor,0.890322,6862.821021
3,SGD Regressor,0.889850,6877.576274
8,XGBoost Regressor,0.880312,7169.176614
1,Random Forest Regressor,0.764775,10050.426046
6,Extra Trees Regressor,0.754468,10268.274665
7,Gradient Boosting Regressor,0.705907,11237.907604
5,K Neighbors Regressor,0.637728,12472.708734
4,Elastic Net Regressor,0.147385,19134.616459
2,Support Vector Regressor,-0.014925,20876.614291


In [32]:
models_fitted

{'Linear Regression Regressor': Pipeline(steps=[('tfidf',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('tfidf1', TfidfVectorizer(),
                                                   'Job Title'),
                                                  ('tfidf2', TfidfVectorizer(),
                                                   'Skills')])),
                 ('run_model', LinearRegression())],
          verbose=True),
 'Random Forest Regressor': Pipeline(steps=[('tfidf',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('tfidf1', TfidfVectorizer(),
                                                   'Job Title'),
                                                  ('tfidf2', TfidfVectorizer(),
                                                   'Skills')])),
                 ('run_model', RandomForestRegressor(n_estimators=150))],
          verbose=True),
 'Support

## Voting Regressor

In [51]:
from sklearn.ensemble import VotingRegressor

In [52]:
top_3_models = dict(sorted(r2_scores.items(), key=lambda x:x[1], reverse=True)[:3])
top_3_models

{'Linear Regression Regressor': 0.8903221296001184,
 'SGD Regressor': 0.88985000177259,
 'XGBoost Regressor': 0.880311555633711}

In [53]:
vr_input = []

for top_model in top_3_models.keys():
    vr_input.append((top_model, models_fitted[top_model]))
                    
vr_input

[('Linear Regression Regressor',
  Pipeline(steps=[('tfidf',
                   ColumnTransformer(remainder='passthrough',
                                     transformers=[('tfidf1', TfidfVectorizer(),
                                                    'Job Title'),
                                                   ('tfidf2', TfidfVectorizer(),
                                                    'Skills')])),
                  ('run_model', LinearRegression())],
           verbose=True)),
 ('SGD Regressor',
  Pipeline(steps=[('tfidf',
                   ColumnTransformer(remainder='passthrough',
                                     transformers=[('tfidf1', TfidfVectorizer(),
                                                    'Job Title'),
                                                   ('tfidf2', TfidfVectorizer(),
                                                    'Skills')])),
                  ('run_model', SGDRegressor(max_iter=10000))],
           verbose=True)),
 ('XGBoo

In [56]:
vr = VotingRegressor(vr_input, verbose=True)
vr

VotingRegressor(estimators=[('Linear Regression Regressor',
                             Pipeline(steps=[('tfidf',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('tfidf1',
                                                                               TfidfVectorizer(),
                                                                               'Job '
                                                                               'Title'),
                                                                              ('tfidf2',
                                                                               TfidfVectorizer(),
                                                                               'Skills')])),
                                             ('run_model', LinearRegression())],
                                      verbose=True)),
                   

In [57]:
vr.fit(X_train, y_train)
vr

[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing run_model, total=   1.6s
[Voting]  (1 of 3) Processing Linear Regression Regressor, total=   1.8s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing run_model, total=   6.7s
[Voting] ............ (2 of 3) Processing SGD Regressor, total=   6.9s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing run_model, total=   0.7s
[Voting] ........ (3 of 3) Processing XGBoost Regressor, total=   0.9s


VotingRegressor(estimators=[('Linear Regression Regressor',
                             Pipeline(steps=[('tfidf',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('tfidf1',
                                                                               TfidfVectorizer(),
                                                                               'Job '
                                                                               'Title'),
                                                                              ('tfidf2',
                                                                               TfidfVectorizer(),
                                                                               'Skills')])),
                                             ('run_model', LinearRegression())],
                                      verbose=True)),
                   

In [59]:
y_pred = vr.predict(X_test)
print('R2 Score:', r2_score(y_test, y_pred))
print('RMSE:', math.sqrt(mean_squared_error(y_test, y_pred)))

R2 Score: 0.9011466368488134
RMSE: 6515.3671360886665


## Saving the model

In [61]:
import joblib

In [62]:
joblib.dump(vr, 'salary_model_30_11.pkl')

['salary_model_30_11.pkl']

In [63]:
joblib_model = joblib.load('salary_model_30_11.pkl')

In [64]:
y_pred = joblib_model.predict(X_test)
print('R2 Score:', r2_score(y_test, y_pred))
print('RMSE:', math.sqrt(mean_squared_error(y_test, y_pred)))

R2 Score: 0.9011466368488134
RMSE: 6515.3671360886665


In [83]:
rec = pd.DataFrame(X_test.iloc[5, :])

In [84]:
rec.T

,Job Title,Skills,Education_ASSOCIATE,Education_BACHELOR,Education_DOCTORATE,Education_HIGH SCHOOL,Education_MASTER,Education_NOT SPECIFIED,Education_VOCATIONAL
6077,Sql Developer,"Sql Server Reporting Services, Extract Transfo...",0,1,0,0,0,0,0


In [87]:
y_cr = vr.predict(rec.T)

In [86]:
y_test.iloc[5, :]

Salary    104000.0
Name: 6077, dtype: float64

In [89]:
y_cr[0]

104031.90106751463